This notebook will be use to explore the metadata of the Fishing Global Watch of a Earth Engine Snippet (GFW/GFF/V1/fishing_hours)

Metadata stands for data about data. In case of images, metadata means details about the image and its production. Some metadata is generated automatically by the capturing device. 



In [3]:
# Import packages
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Authenticate google earth engine
ee.Authenticate()

Enter verification code:  4/1AfgeXvumorzLzTL6s37UhDKL9gLFtiatHrH6hKY2VtABkhQ0551O8w9FHzM



Successfully saved authorization token.


In [4]:
# Initialize google earth engine 
ee.Initialize()

In [5]:
# read in the data on fishing hours from google earth engine
dataset = ee.ImageCollection('GFW/GFF/V1/fishing_hours')
# read in data on total vessel hours from google earth engine
dataset2 = ee.ImageCollection('GFW/GFF/V1/vessel_hours')

In [6]:
dataset

In [7]:
dataset2

In [8]:
# Create a filter for the dates
elnino_data = dataset.filter(ee.Filter.date('2015-01-01', '2016-01-01'))
elnino_data

In [9]:
u_lon = -80
u_lat = -10
x = ee.Geometry.Point([u_lon, u_lat])

In [10]:
# See image properties with their names and values

fishing_image = ee.ImageCollection('GFW/GFF/V1/fishing_hours') \
    .filterBounds(x) \
    .filter(ee.Filter.date('2015-01-01', '2015-01-10')) \
    .first()

In [11]:
fishing_props = geemap.image_props(fishing_image)
fishing_props.getInfo()


{'IMAGE_DATE': '2015-01-01',
 'NOMINAL_SCALE': 1113.1949079327358,
 'country': 'ARG',
 'system:asset_size': '0.015705 MB',
 'system:band_names': ['drifting_longlines',
  'fixed_gear',
  'other_fishing',
  'purse_seines',
  'squid_jigger',
  'trawlers'],
 'system:id': 'GFW/GFF/V1/fishing_hours/ARG-20150101',
 'system:index': 'ARG-20150101',
 'system:time_end': '2015-01-02 00:00:00',
 'system:time_start': '2015-01-01 00:00:00',
 'system:version': 1518618589881092}

In [12]:
country = fishing_props.get('country')
print('Country: ', country.getInfo())

Country:  ARG


In [37]:
properties = fishing_image.propertyNames()
print('Metadata properties: ', properties.getInfo())

Metadata properties:  ['system:time_start', 'country', 'system:footprint', 'system:time_end', 'system:version', 'system:id', 'system:asset_size', 'system:index', 'system:bands', 'system:band_names']


In [31]:
def flag_totals(gee_snipet, lon, lat, flag, start_date, end_date):
    dataset_name = ee.ImageCollection(gee_snipet)
    area = ee.Geometry.Point([lon, lat])
    collection = (dataset_name
                  .filterMetadata('country', 'equals', flag)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.bounds(area))
                  .sum())
    return ee.Image(collection)

flag_totals('GFW/GFF/V1/fishing_hours',-80.5,-10.5, "ARG",'2016-01-01','2017-01-01')

In [34]:
arg = flag_totals('GFW/GFF/V1/fishing_hours',-80.5,-10.5, "ARG",'2016-01-01','2017-01-01')

arg_all = arg.reduce(ee.Reducer.sum())

print(arg_all)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.reduce",
    "arguments": {
      "image": {
        "functionInvocationValue": {
          "functionName": "reduce.sum",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.filter",
                            "arguments": {
                              "collection": {
                                "functionInvocationValue": {
                                  "functionName": "ImageCollection.load",
                                  "arguments": {
                 

In [35]:
arg_drift_lines = arg.select('drifting_longlines')

print(arg_drift_lines)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.select",
    "arguments": {
      "bandSelectors": {
        "constantValue": [
          "drifting_longlines"
        ]
      },
      "input": {
        "functionInvocationValue": {
          "functionName": "reduce.sum",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.filter",
                            "arguments": {
                              "collection": {
                                "functionInvocationValue": {
                                  "fun

In [18]:
# Function to create a df with the band info
def get_info_bands(lat,lon,id,gee_snipet,scale):
    x = ee.ImageCollection(gee_snipet)
    area = ee.Geometry.Point([lon, lat])
    country = x.get(id)
    time_series = country.getRegion(area,scale).getInfo()
    df = pd.DataFrame(time_series)
    headers = df.loc[0]
    df.columns = headers
    df = df.drop(0)
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')
    df[id] = df[id].astype(float)
    print(df.head(),"\n\n")
    print(df[id].describe(),"\n\n")
    list5 = df[id].astype(float).nlargest(5)
    print("The Top 5 Largest values are:\n", list5,"\n\n")
    print(df.iloc[list5.index-1],"\n\n")

    
get_info_bands(-10.5,-80, "country" ,"GFW/GFF/V1/fishing_hours", 1000)

AttributeError: 'ComputedObject' object has no attribute 'getRegion'

In [32]:
#Get the number if images in the region and in the time frame


def get_count_images(lat,lon,gee_snipet,start_date,end_date):
    dataset_name = ee.ImageCollection(gee_snipet)
    area = ee.Geometry.Point([lon, lat])
    collection = (dataset_name
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.bounds(area)))
    first_image = collection.first()
    bands_list = first_image.bandNames().getInfo()
    count = collection.size()
    #print(collection)
    print("Count:", str(count.getInfo())+ '\n')
    print(bands_list)
    
    
get_count_images(-10, -80.2, 'GFW/GFF/V1/fishing_hours', '20015-07-18', '2015-07-20')


EEException: Collection.reduceColumns: Empty date ranges not supported for the current operation.